## Essential Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
from keras.regularizers import l2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
tf.keras.__version__

'2.4.0'

## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(spec images)/temp data(spec images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [299, 299]

In [ ]:
print(num_classes)

2


## Creating a new model

In [ ]:
model = Sequential()
model.add(InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 8, 8, 2048)        21802784  
_________________________________________________________________
flatten_7 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_14 (Dense)             (None, 128)               16777344  
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 129       
Total params: 38,580,257
Trainable params: 16,777,473
Non-trainable params: 21,802,784
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (299, 299),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (299, 299),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
print(test_set)

In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=40,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
  # callbacks=[early]
)

f=open('/content/drive/My Drive/IVP Course Project/Transfer Learning Model/TL-SI-Inceptionv3-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/IVP Course Project/Transfer Learning Model/TL-SI-Inceptionv3-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
300/300 [==============================] - 65s 200ms/step - loss: 3.9353 - accuracy: 0.8103 - mean_absolute_error: 32.6395 - mean_squared_error: 1678.2076 - hinge: 3.9293 - binary_crossentropy: 2.8606 - val_loss: 1.4253 - val_accuracy: 0.8643 - val_mean_absolute_error: 28.9965 - val_mean_squared_error: 1295.7788 - val_hinge: 1.4226 - val_binary_crossentropy: 2.0251
Epoch 2/40
300/300 [==============================] - 58s 194ms/step - loss: 0.7312 - accuracy: 0.9292 - mean_absolute_error: 29.0849 - mean_squared_error: 1174.9150 - hinge: 0.7286 - binary_crossentropy: 1.0211 - val_loss: 1.4510 - val_accuracy: 0.8886 - val_mean_absolute_error: 27.3235 - val_mean_squared_error: 1081.7207 - val_hinge: 1.4488 - val_binary_crossentropy: 1.6518
Epoch 3/40
300/300 [==============================] - 58s 193ms/step - loss: 0.5750 - accuracy: 0.9324 - mean_absolute_error: 27.8238 - mean_squared_error: 1091.1271 - hinge: 0.5729 - binary_crossentropy: 0.9872 - val_loss: 0.4230 - val_accur

## Plots for each epoch

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='test loss')
plt.legend()
plt.savefig('/content/drive/My Drive/IVP Course Project/Transfer Learning Model/Loss-Epoch-plot(plot images)')
plt.show()

# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='test acc')
plt.legend()
plt.savefig('/content/drive/My Drive/IVP Course Project/Transfer Learning Model/Accuracy-Epoch-plot(plot images)')
plt.show()